In [6]:
import whisper
import ffmpeg

/Users/manaadvice/opt/anaconda3/envs/openai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = whisper.load_model("base")

In [8]:
model.device

device(type='cpu')

In [29]:
from IPython.display import Audio
audio_file = "fed_meeting_trimmed.mp3"

In [36]:
def trim_file(input_file, output_file, start_time, end_time):
    (
        ffmpeg
        .input(input_file)
        .filter("atrim", start=start_time, end=end_time)
        .output(output_file)
        .run()
    )
    
input_file = "fed_meeting.mp4"
output_file = "fed_meeting_trimmed.mp4"
start_time = "00:06:20" # start 0 seconds into the input file
end_time = "00:06:40" # end 15 seconds into the input file

trim_file(input_file, output_file, start_time, end_time)

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --

In [38]:
Audio(output_file)

In [40]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(output_file)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions(fp16 = False)
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: en
My colleagues and I are strongly committed to bringing inflation back down to our 2% goal. We have both the tools that we need and the resolve it will take to restore price stability on behalf of American families and businesses. Price stability is the responsibility of the Federal Reserve and serves as the bedrock of our economy.


## Subtitles generator

In [22]:
import subprocess
import os

def video2mp3(video_file, output_ext="mp3"):
    # Convert video file to mp3 format 
    filename, ext = os.path.splitext(video_file)
    subprocess.run(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"

In [41]:
output_file = "fed_meeting_trimmed_audio.mp3"

trim_file(input_file, output_file, start_time, end_time)

audio_file = video2mp3(output_file)

Audio(audio_file)

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --

In [42]:
def translate(audio):
    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file, **translate_options)
    return result

In [43]:
result = translate(audio_file)
print(result["text"])

/Users/manaadvice/opt/anaconda3/envs/openai/lib/python3.10/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 My colleagues and I are strongly committed to bringing inflation back down to our 2% goal. We have both the tools that we need and the resolve it will take to restore price stability on behalf of American families and businesses. Price stability is the responsibility of the Federal Reserve and serves as the bedrock of our economy.


In [44]:
from whisper.utils import write_vtt

output_dir = "content/"
audio_filename = audio_file.split(".")

with open(os.path.join(output_dir, audio_filename + ".vtt"), "w") as vtt:
    write_vtt(result["segments"], file=vtt)

ImportError: cannot import name 'write_vtt' from 'whisper.utils' (/Users/manaadvice/opt/anaconda3/envs/openai/lib/python3.10/site-packages/whisper/utils.py)

In [ ]:
subtitle = audio_filename + ".vtt"
output_video = audio_filename + "_subtitled.mp4"

os.system(f"ffmpeg -i {input_file} -vf subtitles={subtitle} {output_video}")

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(output_video, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
    <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Speech-To-Text Transcription

Speech-to-Text Python Project to convert English Audio to English Text 

In [45]:
!wget -O audio.mp3 http://www.moviesoundclips.net/movies1/batmanbegins/bats.mp3

/bin/bash: wget: command not found


In [51]:
output_dir = "content/"
audio_filename = "bats.mp3"
audio_file = os.path.join(output_dir, audio_filename)
print(audio_file)

Audio(audio_file)

content/bats.mp3


In [52]:
result = model.transcribe(audio_file)
print(result["text"])

/Users/manaadvice/opt/anaconda3/envs/openai/lib/python3.10/site-packages/whisper/transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 That's your knob, Colonel. That's my beat. Believe in the billionaire playboy. That's where your clock is pushing.


## Automatic Podcast Transcriptor